In [0]:
-----------------------FEATURE ENGINEERING-----------------------------
-- Create session-level features table
CREATE SCHEMA IF NOT EXISTS workspace.gold;

CREATE OR REPLACE TABLE workspace.gold.session_metrics AS
SELECT 
  user_session,
  user_id,
  
  -- Time features
  MIN(event_time) as session_start,
  MAX(event_time) as session_end,
  TIMESTAMPDIFF(MINUTE, MIN(event_time), MAX(event_time)) as session_duration_minutes,
  DATE(MIN(event_time)) as session_date,
  HOUR(MIN(event_time)) as session_start_hour,
  DAYOFWEEK(MIN(event_time)) as session_day_of_week,
  
  -- Event counts
  COUNT(*) as total_events,
  SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) as views_count,
  SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as cart_adds_count,
  SUM(CASE WHEN event_type = 'remove_from_cart' THEN 1 ELSE 0 END) as cart_removes_count,
  SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as purchases_count,
  
  -- Unique products
  COUNT(DISTINCT product_id) as unique_products_viewed,
  COUNT(DISTINCT CASE WHEN event_type = 'cart' THEN product_id END) as unique_products_carted,
  COUNT(DISTINCT CASE WHEN event_type = 'purchase' THEN product_id END) as unique_products_purchased,
  
  -- Revenue
  SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) as total_revenue,
  AVG(CASE WHEN event_type = 'purchase' THEN price END) as avg_purchase_price,
  
  -- Conversion flags
  MAX(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as has_cart_event,
  MAX(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as has_purchase_event,
  
  -- Funnel position
  CASE 
    WHEN MAX(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) = 1 THEN 'converted'
    WHEN MAX(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) = 1 THEN 'abandoned_cart'
    ELSE 'browsing_only'
  END as session_funnel_stage

FROM workspace.silver.events_cleaned
GROUP BY user_session, user_id;

-- Quick check of session features
SELECT 
  COUNT(*) as total_sessions,
  ROUND(AVG(session_duration_minutes), 2) as avg_session_duration_min,
  ROUND(AVG(total_events), 2) as avg_events_per_session,
  ROUND(AVG(unique_products_viewed), 2) as avg_products_viewed,
  ROUND(SUM(total_revenue), 2) as total_platform_revenue
FROM workspace.gold.session_metrics;

-- Session funnel breakdown
SELECT 
  session_funnel_stage,
  COUNT(*) as session_count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage,
  ROUND(AVG(session_duration_minutes), 1) as avg_duration_min,
  ROUND(AVG(unique_products_viewed), 1) as avg_products_viewed
FROM workspace.gold.session_metrics
GROUP BY session_funnel_stage
ORDER BY session_count DESC;


-- Create user-level features table
CREATE OR REPLACE TABLE workspace.gold.user_features AS
SELECT 
  user_id,
  
  -- Account activity
  MIN(event_time) as first_activity_date,
  MAX(event_time) as last_activity_date,
  DATEDIFF(MAX(event_time), MIN(event_time)) as customer_lifetime_days,
  
  -- Session behavior
  COUNT(DISTINCT user_session) as total_sessions,
  COUNT(DISTINCT DATE(event_time)) as active_days,
  
  -- Event totals
  COUNT(*) as total_events,
  SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) as total_views,
  SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as total_cart_adds,
  SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as total_purchases,
  
  -- Purchase behavior
  SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) as lifetime_revenue,
  AVG(CASE WHEN event_type = 'purchase' THEN price END) as avg_order_value,
  COUNT(DISTINCT CASE WHEN event_type = 'purchase' THEN user_session END) as purchase_sessions,
  
  -- Product diversity
  COUNT(DISTINCT product_id) as unique_products_interacted,
  COUNT(DISTINCT category_code) as unique_categories_browsed,
  COUNT(DISTINCT brand) as unique_brands_browsed,
  
  -- User classification
  CASE 
    WHEN SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) > 0 THEN 'buyer'
    WHEN SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) > 0 THEN 'cart_abandoner'
    ELSE 'browser'
  END as user_type,
  
  CASE 
    WHEN SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) >= 3 THEN 'repeat_buyer'
    WHEN SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) > 0 THEN 'one_time_buyer'
    ELSE 'non_buyer'
  END as buyer_segment

FROM workspace.silver.events_cleaned
GROUP BY user_id;

-- User segmentation overview
SELECT 
  user_type,
  buyer_segment,
  COUNT(*) as user_count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as pct_of_users,
  ROUND(AVG(lifetime_revenue), 2) as avg_lifetime_value,
  ROUND(AVG(total_sessions), 1) as avg_sessions
FROM workspace.gold.user_features
GROUP BY user_type, buyer_segment
ORDER BY user_count DESC;

-- Create product-level features table
CREATE OR REPLACE TABLE workspace.gold.product_features AS
SELECT 
  product_id,
  MAX(category_id) as category_id,
  MAX(category_code) as category_code,
  MAX(brand) as brand,
  AVG(price) as avg_price,
  
  -- Engagement metrics
  COUNT(*) as total_interactions,
  SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) as view_count,
  SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as cart_add_count,
  SUM(CASE WHEN event_type = 'remove_from_cart' THEN 1 ELSE 0 END) as cart_remove_count,
  SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as purchase_count,
  
  -- Unique users
  COUNT(DISTINCT user_id) as unique_viewers,
  COUNT(DISTINCT CASE WHEN event_type = 'cart' THEN user_id END) as unique_cart_users,
  COUNT(DISTINCT CASE WHEN event_type = 'purchase' THEN user_id END) as unique_buyers,
  
  -- Revenue
  SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) as total_revenue,
  
  -- Conversion rates
  ROUND(SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) * 100.0 / 
        NULLIF(SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END), 0), 2) as view_to_cart_rate,
  ROUND(SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) * 100.0 / 
        NULLIF(SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END), 0), 2) as cart_to_purchase_rate,
  ROUND(SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) * 100.0 / 
        NULLIF(SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END), 0), 2) as view_to_purchase_rate,
  
  -- Product flags
  CASE WHEN SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) = 0 THEN 1 ELSE 0 END as never_purchased

FROM workspace.silver.events_cleaned
GROUP BY product_id;


-- Top performing products by revenue
SELECT 
  product_id,
  brand,
  category_code,
  ROUND(avg_price, 2) as price,
  view_count,
  cart_add_count,
  purchase_count,
  ROUND(total_revenue, 2) as revenue,
  view_to_cart_rate,
  cart_to_purchase_rate,
  view_to_purchase_rate
FROM workspace.gold.product_features
WHERE purchase_count > 0
ORDER BY total_revenue DESC
LIMIT 20;


-- Category performance summary
CREATE OR REPLACE TABLE workspace.gold.category_performance AS
SELECT 
  category_code,
  COUNT(DISTINCT product_id) as product_count,
  SUM(view_count) as total_views,
  SUM(cart_add_count) as total_cart_adds,
  SUM(purchase_count) as total_purchases,
  SUM(total_revenue) as category_revenue,
  ROUND(AVG(avg_price), 2) as avg_category_price,
  ROUND(AVG(view_to_purchase_rate), 2) as avg_conversion_rate,
  COUNT(DISTINCT CASE WHEN never_purchased = 1 THEN product_id END) as dead_stock_products
FROM workspace.gold.product_features
GROUP BY category_code
ORDER BY category_revenue DESC;


-- Brand performance summary
CREATE OR REPLACE TABLE workspace.gold.brand_performance AS
SELECT 
  brand,
  COUNT(DISTINCT product_id) as product_count,
  SUM(view_count) as total_views,
  SUM(cart_add_count) as total_cart_adds,
  SUM(purchase_count) as total_purchases,
  SUM(total_revenue) as brand_revenue,
  ROUND(AVG(avg_price), 2) as avg_brand_price,
  ROUND(AVG(view_to_purchase_rate), 2) as avg_conversion_rate
FROM workspace.gold.product_features
GROUP BY brand
ORDER BY brand_revenue DESC;

-- Check if table exists and has data

-- Preview top products
SELECT * FROM workspace.gold.product_features ORDER BY total_revenue DESC LIMIT 10;
-- Top 10 categories
SELECT * FROM workspace.gold.category_performance LIMIT 10;


-- Top 10 brands
SELECT * FROM workspace.gold.brand_performance LIMIT 10;




-----------KEY BUSINESS FINDINGS------------------
-- Create KPI summary table for dashboards
CREATE OR REPLACE TABLE workspace.gold.platform_kpis AS
SELECT 
  -- Revenue metrics
  SUM(total_revenue) as total_revenue,
  COUNT(DISTINCT user_id) as total_users,
  SUM(purchases_count) as total_orders,
  SUM(total_revenue) / COUNT(DISTINCT user_id) as revenue_per_user,
  SUM(total_revenue) / SUM(purchases_count) as average_order_value,
  
  -- Conversion metrics
  COUNT(DISTINCT CASE WHEN has_purchase_event = 1 THEN user_id END) as buyers,
  COUNT(DISTINCT CASE WHEN has_purchase_event = 1 THEN user_id END) * 100.0 / COUNT(DISTINCT user_id) as user_conversion_rate,
  
  SUM(CASE WHEN has_purchase_event = 1 THEN 1 ELSE 0 END) as converted_sessions,
  SUM(CASE WHEN has_purchase_event = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as session_conversion_rate,
  
  SUM(CASE WHEN has_cart_event = 1 AND has_purchase_event = 0 THEN 1 ELSE 0 END) as abandoned_carts,
  SUM(CASE WHEN has_cart_event = 1 AND has_purchase_event = 0 THEN 1 ELSE 0 END) * 100.0 / 
    NULLIF(SUM(CASE WHEN has_cart_event = 1 THEN 1 ELSE 0 END), 0) as cart_abandonment_rate,
  
  -- Engagement metrics
  COUNT(*) as total_sessions,
  AVG(session_duration_minutes) as avg_session_duration,
  AVG(unique_products_viewed) as avg_products_per_session,
  SUM(total_events) as total_events

FROM workspace.gold.session_metrics;

-- the KPIs
SELECT * FROM workspace.gold.platform_kpis;


-- Daily performance trends
CREATE OR REPLACE TABLE workspace.gold.daily_trends AS
SELECT 
  session_date,
  DAYNAME(session_date) as day_name,
  
  -- Revenue
  SUM(total_revenue) as daily_revenue,
  
  -- Sessions & Users
  COUNT(DISTINCT user_session) as sessions,
  COUNT(DISTINCT user_id) as active_users,
  
  -- Orders
  SUM(purchases_count) as orders,
  SUM(total_revenue) / NULLIF(SUM(purchases_count), 0) as aov,
  
  -- Conversion
  SUM(CASE WHEN has_purchase_event = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as conversion_rate,
  
  -- Engagement
  AVG(session_duration_minutes) as avg_session_duration,
  AVG(unique_products_viewed) as avg_products_viewed

FROM workspace.gold.session_metrics
GROUP BY session_date
ORDER BY session_date;

-- View daily trends
SELECT * FROM workspace.gold.daily_trends ORDER BY session_date DESC LIMIT 10;


-- Conversion funnel breakdown
SELECT 
  'Total Sessions' as stage,
  COUNT(*) as count,
  100.0 as percentage,
  0 as drop_off_rate
FROM workspace.gold.session_metrics

UNION ALL

SELECT 
  'Sessions with Views' as stage,
  SUM(CASE WHEN views_count > 0 THEN 1 ELSE 0 END) as count,
  ROUND(SUM(CASE WHEN views_count > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as percentage,
  ROUND(100 - (SUM(CASE WHEN views_count > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) as drop_off_rate
FROM workspace.gold.session_metrics

UNION ALL

SELECT 
  'Sessions with Cart Adds' as stage,
  SUM(CASE WHEN cart_adds_count > 0 THEN 1 ELSE 0 END) as count,
  ROUND(SUM(CASE WHEN cart_adds_count > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as percentage,
  ROUND(100 - (SUM(CASE WHEN cart_adds_count > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) as drop_off_rate
FROM workspace.gold.session_metrics

UNION ALL

SELECT 
  'Sessions with Purchases' as stage,
  SUM(CASE WHEN purchases_count > 0 THEN 1 ELSE 0 END) as count,
  ROUND(SUM(CASE WHEN purchases_count > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as percentage,
  ROUND(100 - (SUM(CASE WHEN purchases_count > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2) as drop_off_rate
FROM workspace.gold.session_metrics

ORDER BY count DESC;

-- RFM-style user segmentation
SELECT 
  buyer_segment,
  user_type,
  COUNT(*) as user_count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as pct_users,
  ROUND(AVG(lifetime_revenue), 2) as avg_ltv,
  ROUND(AVG(total_sessions), 1) as avg_sessions,
  ROUND(AVG(total_purchases), 1) as avg_purchases,
  ROUND(SUM(lifetime_revenue), 2) as total_segment_revenue,
  ROUND(SUM(lifetime_revenue) * 100.0 / SUM(SUM(lifetime_revenue)) OVER(), 2) as pct_revenue

FROM workspace.gold.user_features
GROUP BY buyer_segment, user_type
ORDER BY total_segment_revenue DESC;


-- Best performing products (high conversion)
SELECT 
  product_id,
  brand,
  category_code,
  ROUND(avg_price, 2) as price,
  view_count,
  purchase_count,
  ROUND(total_revenue, 2) as revenue,
  view_to_purchase_rate as conversion_rate,
  ROUND(total_revenue / NULLIF(purchase_count, 0), 2) as revenue_per_sale

FROM workspace.gold.product_features
WHERE purchase_count >= 100  -- Filter for statistical significance
ORDER BY view_to_purchase_rate DESC
LIMIT 20;


-- Worst performing products (low conversion, high views)
SELECT 
  product_id,
  brand,
  category_code,
  ROUND(avg_price, 2) as price,
  view_count,
  cart_add_count,
  purchase_count,
  view_to_purchase_rate as conversion_rate

FROM workspace.gold.product_features
WHERE view_count >= 1000  -- Products with significant traffic
  AND purchase_count = 0  -- But no sales
ORDER BY view_count DESC
LIMIT 20;

-- Peak shopping hours
CREATE OR REPLACE TABLE workspace.gold.hourly_patterns AS
SELECT 
  session_start_hour as hour,
  COUNT(*) as sessions,
  SUM(total_revenue) as revenue,
  ROUND(AVG(total_revenue), 2) as avg_revenue_per_session,
  SUM(purchases_count) as orders,
  ROUND(SUM(CASE WHEN has_purchase_event = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as conversion_rate
FROM workspace.gold.session_metrics
GROUP BY session_start_hour
ORDER BY session_start_hour;

select *
from workspace.gold.hourly_patterns

-- Day of week pattern analysis
CREATE OR REPLACE TABLE workspace.gold.dow_patterns AS
SELECT 
  session_day_of_week as day_of_week,
  CASE session_day_of_week
    WHEN 1 THEN 'Sunday'
    WHEN 2 THEN 'Monday'
    WHEN 3 THEN 'Tuesday'
    WHEN 4 THEN 'Wednesday'
    WHEN 5 THEN 'Thursday'
    WHEN 6 THEN 'Friday'
    WHEN 7 THEN 'Saturday'
  END as day_name,
  COUNT(*) as sessions,
  SUM(total_revenue) as revenue,
  SUM(purchases_count) as orders,
  ROUND(SUM(CASE WHEN has_purchase_event = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as conversion_rate
FROM workspace.gold.session_metrics
GROUP BY session_day_of_week
ORDER BY session_day_of_week;